In [57]:
import numpy as np
import pandas as pd

import data_utils

In [58]:
df = data_utils.etl()

In [59]:
current_data = df[df['ingestion_ts'] == max(df['ingestion_ts'])]
sorted_data = current_data.sort_values(by="score", ascending=False)
sorted_data[(sorted_data['price'] > 100) * (sorted_data['price'] < 200)].head(20)

,price,score,deal,num_tickets,section,title,game_ts,ingestion_ts,row
1,183,10.0,Amazing Deal,1-3,Luxury Suite 47,Miami Heat at Philadelphia 76ers,2023-02-28T00:00:00,2023-01-30-13-33-25-566172,NA
0,183,10.0,Amazing Deal,12,Luxury Suite 47,Miami Heat at Philadelphia 76ers,2023-02-28T00:00:00,2023-01-30-13-33-25-566172,NA
14,148,9.9,Amazing Deal,2,101,Orlando Magic at Philadelphia 76ers,2023-02-02T00:00:00,2023-01-30-13-33-25-566172,11
64,103,9.8,Amazing Deal,2,119,Orlando Magic at Philadelphia 76ers,2023-02-02T00:00:00,2023-01-30-13-33-25-566172,1
4,144,9.8,Amazing Deal,2,110,New York Knicks at Philadelphia 76ers,2023-02-11T00:00:00,2023-01-30-13-33-25-566172,17
5,169,9.8,Amazing Deal,2,Club Box 15,New York Knicks at Philadelphia 76ers,2023-02-11T00:00:00,2023-01-30-13-33-25-566172,4
2,105,9.8,Amazing Deal,1-3,110,Cleveland Cavaliers at Philadelphia 76ers,2023-02-16T00:00:00,2023-01-30-13-33-25-566172,11
3,103,9.8,Amazing Deal,1-3,104,Cleveland Cavaliers at Philadelphia 76ers,2023-02-16T00:00:00,2023-01-30-13-33-25-566172,11
42,136,9.8,Amazing Deal,1-4,115,Orlando Magic at Philadelphia 76ers,2023-02-02T00:00:00,2023-01-30-13-33-25-566172,1
1,144,9.7,Amazing Deal,1,108,Dallas Mavericks at Philadelphia 76ers,2023-03-29T23:30:00,2023-01-30-13-33-25-566172,7
